# Testing Mantid's CylindricalAbsorption wrapper
<div class="alert alert-block alert-info">
<b>Information:</b> This notebook shows how to use Mantid's CylindricalAbsorption algorithm with a `scipp` Dataset.

<b>Requirements:</b> To run this notebook, you need `mantid`, `scipp` and `numpy` installed as well as the Python script `absorption.py` placed in the same folder as this notebook.   
Additionally, the test file `wish_test.nxs` should reside in the same directory.
</div>

In [1]:
import numpy as np
import scipp as sc
import mantid.simpleapi as simpleapi
from scipp.compat.mantid import load
from absorption import absorption_correction


/home/piotr/anaconda3/envs/skip/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Loading data

Load a suitable dataset first. This is a **HUGE** file so each operation will be quite slow.
The load operation takes ~8s on the DMSC workstation.

In [2]:
filename = "wish_test.nxs"
dataset = sc.compat.mantid.load(filename=filename)

The file is loaded as a `DataArray` with following sizes:

In [3]:
print(dataset.dims)
print(dataset.shape)

[Dim.Spectrum, Dim.Tof]
[194560]


## Defining absorption parameters

Our absorption should be calculated for the sample.

In [4]:
scatter_from = "Sample"

## Calculating absorption

Now we are ready to calculate the correction, passing the sample definition and example cylinder dimensions to the calculator.
We need these, since the data file contains no sample definition.
This calculation takes some time.

In [12]:
correction = absorption_correction(dataset,
                                   ScatterFrom=scatter_from,
                                   CylinderSampleHeight=3,
                                   CylinderSampleRadius=10)

The correction dataset uses wavelengths units, so we need to convert it back to time-of-flight, as in the original sample.

In [13]:
correction = sc.neutron.convert(correction, 'wavelength', 'tof')

The absorption correction has been calculated to match the input dataset over bins ranging from

In [14]:
correction.coords['spectrum'].values[0]

6

to

In [15]:
correction.coords['spectrum'].values[-1]

389125

## More complex parameter definition

We may also define other parameters required for Mantid's algorithm.

In [9]:
sample_number_density = 0.07192
number_wavelength_points = 5
cylinder_sample_height = 4
cylinder_sample_radius = 0.4
number_of_slices = 2

These parameters can just be appended to the `absorption_correction` call.

In [10]:
correction = absorption_correction(dataset,
                                   ScatterFrom=scatter_from,
                                   SampleNumberDensity=sample_number_density,
                                   NumberOfWavelengthPoints=number_wavelength_points,
                                   CylinderSampleHeight=cylinder_sample_height,
                                   CylinderSampleRadius=cylinder_sample_radius,
                                   NumberOfSlices=number_of_slices)

In [11]:
correction = sc.neutron.convert(correction, Dim.Wavelength, Dim.Tof)
print(correction.coords['spectrum'].values[0])
print(correction.coords['spectrum'].values[-1])

6
389125


Adventurous users might want to visualize the correction itself.

In [ ]:
from scipp.plot import plot

In [ ]:
plot(correction)